<a href="https://colab.research.google.com/github/garthajon/QuantFinanceIntro/blob/main/LIQUIDITY-SCREENER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import time
import requests
import pandas as pd
import yfinance as yf
from tqdm import tqdm

# -----------------------------
# Step 1: Scrape S&P 500 tickers from Wikipedia
# -----------------------------
wiki_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

# Pretend to be a browser to avoid 403
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    )
}

response = requests.get(wiki_url, headers=headers)
if response.status_code != 200:
    raise Exception(f"Error fetching Wikipedia page: {response.status_code}")

tables = pd.read_html(response.text)
sp500_df = tables[0]  # first table has the tickers
tickers = sp500_df["Symbol"].tolist()
print(f"Loaded {len(tickers)} S&P 500 tickers")

# -----------------------------
# Screening parameters
# -----------------------------
lookback_years = 2
rolling_window = 12  # months
min_data_months = rolling_window + 1

# Containers for screened tickers
healthy_trend = []
explosive_rally = []

# -----------------------------
# Helper function: compute monthly z-scores
# -----------------------------
def compute_monthly_zscores(ticker):
    try:
        df = yf.download(ticker, period=f"{lookback_years}y", auto_adjust=True, progress=False)
        if df.empty:
            return None
    except Exception:
        return None

    # Flatten multi-index if present
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)

    df.index = pd.to_datetime(df.index)
    df = df.sort_index()

    # Daily spread
    df["spread_ret"] = (df["High"] - df["Low"]) / df["Close"].shift(1)

    # Monthly aggregation
    monthly = pd.DataFrame({
        "close": df["Close"].resample("ME").last(),
        "volume": df["Volume"].resample("ME").mean(),
        "spread": df["spread_ret"].resample("ME").mean()
    }).dropna()

    if monthly.shape[0] < min_data_months:
        return None

    # Returns
    monthly["returns"] = monthly["close"].pct_change()

    # Rolling prior-year stats
    r_ret = monthly["returns"].shift(1)
    r_vol = monthly["volume"].shift(1)
    r_spr = monthly["spread"].shift(1)

    monthly["ret_z"] = (monthly["returns"] - r_ret.rolling(rolling_window).mean()) / r_ret.rolling(rolling_window).std()
    monthly["vol_z"] = (monthly["volume"] - r_vol.rolling(rolling_window).mean()) / r_vol.rolling(rolling_window).std()
    monthly["spr_z"] = (monthly["spread"] - r_spr.rolling(rolling_window).mean()) / r_spr.rolling(rolling_window).std()

    monthly = monthly.dropna()
    if monthly.shape[0] < rolling_window:
        return None

    return monthly

# -----------------------------
# Step 2: Loop through tickers and screen
# -----------------------------
for ticker in tqdm(tickers):
    monthly = compute_monthly_zscores(ticker)
    time.sleep(0.5)  # modest rate control to avoid yfinance throttling

    if monthly is None:
        continue

    latest = monthly.iloc[-1]  # last month z-scores
    ret_z = latest["ret_z"]
    vol_z = latest["vol_z"]
    spr_z = latest["spr_z"]

    # Scenario 1: Healthy Trend (Return↑, Volume↑, Spread↓)
    if (ret_z > 0) and (vol_z > 0) and (spr_z < 0):
        healthy_trend.append(ticker)

    # Scenario 2: Explosive Rally (Return↑, Volume↑, Spread↑)
    if (ret_z > 0) and (vol_z > 0) and (spr_z > 0):
        explosive_rally.append(ticker)

# -----------------------------
# Step 3: Output results
# -----------------------------
print("\nStocks meeting Healthy Trend (Return↑, Volume↑, Spread↓):")
print(healthy_trend)

print("\nStocks meeting Explosive Rally (Return↑, Volume↑, Spread↑):")
print(explosive_rally)


/tmp/ipython-input-1571622045.py:25: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


Loaded 503 S&P 500 tickers


 12%|█▏        | 61/503 [00:48<06:03,  1.22it/s]ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BRK.B']: YFPricesMissingError('possibly delisted; no price data found  (period=2y) (Yahoo error = "No data found, symbol may be delisted")')
 15%|█▌        | 76/503 [01:01<05:35,  1.27it/s]ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (period=2y)')
100%|██████████| 503/503 [06:39<00:00,  1.26it/s]


Stocks meeting Healthy Trend (Return↑, Volume↑, Spread↓):
['ALGN', 'AMT', 'AWK', 'AMP', 'AME', 'ADI', 'ACGL', 'ARES', 'ADP', 'AXON', 'BAX', 'BA', 'BR', 'CMG', 'CFG', 'CAG', 'STZ', 'CPRT', 'CPAY', 'DAY', 'DECK', 'DXCM', 'DOV', 'EOG', 'EPAM', 'EFX', 'EQR', 'ERIE', 'EXPD', 'EXR', 'FAST', 'FCX', 'GRMN', 'GIS', 'HBAN', 'ICE', 'KVUE', 'KDP', 'KEY', 'KEYS', 'LII', 'LOW', 'MMC', 'MOH', 'MSI', 'NDAQ', 'NSC', 'NCLH', 'OKE', 'PAYX', 'PAYC', 'PYPL', 'PEP', 'POOL', 'PSA', 'ROP', 'ROST', 'SPGI', 'TROW', 'UNP', 'DIS', 'WBD']

Stocks meeting Explosive Rally (Return↑, Volume↑, Spread↑):
['ACN', 'ADBE', 'AIG', 'AVY', 'BKNG', 'BMY', 'CHTR', 'CB', 'CHD', 'CMCSA', 'COO', 'DE', 'DG', 'DLTR', 'DASH', 'EQIX', 'FFIV', 'FDS', 'FITB', 'FISV', 'GPN', 'HPE', 'HD', 'HRL', 'HPQ', 'INTU', 'IVZ', 'JBHT', 'JKHY', 'J', 'LIN', 'LYV', 'LKQ', 'LULU', 'MCD', 'MCHP', 'TAP', 'MOS', 'NXPI', 'ODFL', 'OMC', 'PCAR', 'RCL', 'CRM', 'SLB', 'SW', 'LUV', 'SNPS', 'TDY', 'TSN', 'ULTA', 'URI', 'V', 'WMT', 'WY', 'WTW']
